In [1]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [3]:
dataset = load_dataset("wikitext","wikitext-2-raw-v1")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
  return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [5]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [6]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    report_to=[],
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=50,
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

/tmp/ipython-input-4152528388.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.194200,3.378386
2,2.833000,3.376466


TrainOutput(global_step=36718, training_loss=2.929967636415087, metrics={'train_runtime': 7215.2626, 'train_samples_per_second': 10.178, 'train_steps_per_second': 5.089, 'total_flos': 4797060415488000.0, 'train_loss': 2.929967636415087, 'epoch': 2.0})

In [33]:


# Seed text to start generation
seed_text = "how"

# Encode the seed text and move to the same device as the model
inputs = tokenizer.encode(seed_text, return_tensors="pt").to(model.device)

# Move the model to the GPU if it's not already there
model.to('cuda')

# Generate text
outputs = model.generate(
    inputs,
    max_length=200,          # total length of output (seed + generated)
    do_sample=True,          # enable sampling (not greedy)
    temperature=0.7,         # creativity (0.7-1.0 works well)
    top_k=40,                # top-k sampling
    top_p=0.9,              # nucleus sampling
    num_return_sequences=1,  # number of outputs
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.5   # discourages repeating the same words
)

# Decode generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print result
print("=== Generated Text ===\n")
print(generated_text)



=== Generated Text ===

how is the structure of a nucleotide sequence ? The answer to this question depends on how it was prepared , what its function has been determined and where in that protein 's DNA sequences have evolved . 
 = Structure ( nuclear envelope ) information processing system – the process by which proteins are assembled from amino acids into complexes consisting mainly of many or all elements ; and Protein folding / translation : as an adaptation for assembly between multiple conformations leading up towards specific foldings within a single protein . In general terms there should be no structural difference in any given protein ' s structure when compared with other proteins except those involved specifically through interaction signaling . However some structures may become more tightly coupled after folding than others because their binding sites are altered during synthesis , so even though certain proteins can bind to certain receptors they do not necessarily bin

In [34]:
model.save_pretrained("gpt2_finetuned")
tokenizer.save_pretrained("gpt2_finetuned")

('gpt2_finetuned/tokenizer_config.json',
 'gpt2_finetuned/special_tokens_map.json',
 'gpt2_finetuned/vocab.json',
 'gpt2_finetuned/merges.txt',
 'gpt2_finetuned/added_tokens.json')

In [37]:
!zip -r gpt2_finetuned.zip gpt2_finetuned

  adding: gpt2_finetuned/ (stored 0%)
  adding: gpt2_finetuned/model.safetensors (deflated 7%)
  adding: gpt2_finetuned/config.json (deflated 52%)
  adding: gpt2_finetuned/tokenizer_config.json (deflated 56%)
  adding: gpt2_finetuned/special_tokens_map.json (deflated 74%)
  adding: gpt2_finetuned/merges.txt (deflated 53%)
  adding: gpt2_finetuned/generation_config.json (deflated 31%)
  adding: gpt2_finetuned/vocab.json (deflated 68%)


In [38]:
from google.colab import files
files.download("gpt2_finetuned.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>